In [1]:
# pip install torch

In [1]:
import pandas as pd
import numpy as np
import copy
from argparse import Namespace
from copy import deepcopy
import torch.nn as nn
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import shutil
import torch.nn.functional as F
import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from typing import Dict, List, OrderedDict, Tuple
import pickle
from collections import OrderedDict
from typing import OrderedDict
from typing import Dict, List, OrderedDict, Tuple, Union
from torch.utils.data import DataLoader, Subset, random_split
from collections import OrderedDict
import os
import random
from sklearn.metrics import precision_score, recall_score

from path import Path
from rich.console import Console
from rich.progress import track
from collections import OrderedDict
import os
import torch
import random
from collections import OrderedDict
from typing import List, OrderedDict, Tuple


In [2]:
import os
import shutil

old_model_path = "D:/Download/pythonProject/HiWi/pFedLA_Folktable/temp/my_model/"
client_models_path = "D:/Download/pythonProject/HiWi/pFedLA_Folktable/src/pkl_files/hn/"


for file in os.listdir(old_model_path):
    file_path = os.path.join(old_model_path, file)
    os.remove(file_path)
    print(f"Deleted: {file_path}")
    
for file in os.listdir(client_models_path):
    file_path = os.path.join(client_models_path, file)
    os.remove(file_path)
    print(f"Deleted: {file_path}")
        


print("All files removed successfully.")


Deleted: D:/Download/pythonProject/HiWi/pFedLA_Folktable/temp/my_model/epoch.pkl
Deleted: D:/Download/pythonProject/HiWi/pFedLA_Folktable/temp/my_model/global_model.pt
All files removed successfully.


In [3]:

def fix_random_seed(seed: int) -> None:
    
#     print("fix_random_seed Chirag")
    
    torch.cuda.empty_cache()
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

    
def clone_parameters(
    src: Union[OrderedDict[str, torch.Tensor], torch.nn.Module]
) -> OrderedDict[str, torch.Tensor]:
    
#     print("clone_parameters Chirag")
    
    if isinstance(src, OrderedDict):
        return OrderedDict(
            {
                name: param.clone().detach().requires_grad_(param.requires_grad)
                for name, param in src.items()
            }
        )
    if isinstance(src, torch.nn.Module):
        return OrderedDict(
            {
                name: param.clone().detach().requires_grad_(param.requires_grad)
                for name, param in src.state_dict(keep_vars=True).items()
            }
        )

    
    
def get_dataloader(
    dataset: str,
    client_id: int,
    batch_size=10,
    valset_ratio=0.2,
    testset_ratio=0.1,
    only_dataset=False,
) -> Dict[str, Union[DataLoader, Subset]]:
    
    print("get_dataloader Chirag ")
    
    args_dict = json.load(open(DATASETS_DIR / "args.json", "r"))
    client_num_in_each_pickles = args_dict["client_num_in_each_pickles"]
    pickles_dir = DATASETS_DIR / dataset / "pickles"
    if os.path.isdir(pickles_dir) is False:
        raise RuntimeError("Please preprocess and create pickles first.")

    pickle_path = (
        pickles_dir / f"{math.floor(client_id / client_num_in_each_pickles)}.pkl"
    )
    with open(pickle_path, "rb") as f:
        subset = pickle.load(f)
    client_dataset = subset[client_id % client_num_in_each_pickles]
    val_samples_num = int(len(client_dataset) * valset_ratio)
    test_samples_num = int(len(client_dataset) * testset_ratio)
    train_samples_num = len(client_dataset) - val_samples_num - test_samples_num
    trainset, valset, testset = random_split(
        client_dataset, [train_samples_num, val_samples_num, test_samples_num]
    )
    if only_dataset:
        return {"train": trainset, "val": valset, "test": testset}
    trainloader = DataLoader(trainset, batch_size, shuffle=True)
    valloader = DataLoader(valset, batch_size)
    testloader = DataLoader(testset, batch_size)
    
    return {"train": trainloader, "val": valloader, "test": testloader}


In [4]:

class ClientBase:
    def __init__(
        self,
        backbone: torch.nn.Module,
        dataset: str,
        batch_size: int,
        valset_ratio: float,
        testset_ratio: float,
        local_epochs: int,
        local_lr: float,
        logger: Console,
        gpu: int,
    ):
        self.device = torch.device(
            "cuda" if gpu and torch.cuda.is_available() else "cpu"
        )
        self.client_id: int = None
        self.valset: DataLoader = None
        self.trainset: DataLoader = None
        self.testset: DataLoader = None
        
        
        # need to change
#         self.model: torch.nn.Module = init_model
        self.model: torch.nn.Module = deepcopy(backbone)
        
#         print("model client", self.model.parameters())
        
#         self.optimizer: torch.optim.Optimizer = torch.optim.SGD(
#             self.model.parameters(), lr=local_lr
#         )
            
        self.optimizer: torch.optim.Optimizer = torch.optim.Adam(
            self.model.parameters(), lr=local_lr
        )
          
        self.batch_size = batch_size
        self.valset_ratio = valset_ratio
        self.testset_ratio = testset_ratio
        self.local_epochs = local_epochs
        self.local_lr = local_lr
        self.criterion = torch.nn.BCELoss()
        self.logger = logger
        
        print("Base Client constructor")

#     @torch.no_grad()
#     def evaluate(self):
#         self.model.eval()
#         size = 0
#         loss = 0
#         correct = 0
#         for x, y in self.testset:
#             x, y = x.to(self.device), y.to(self.device)
            
#             # check this fun
            
#             logits = self.model(x)
                 
#             loss += self.criterion(logits, y)

#             pred = torch.softmax(logits, -1).argmax(-1)

#             correct += (pred == y).int().sum()

#             size += y.size(-1)
        
#         print("correct: %f\n" % (correct))
#         print("loss: %f\n" % (loss))
#         print("size: %f\n" % (size))
        
#         acc = correct / size * 100.0
        
#         print("Acc: %f\n" % (acc))
        
#         loss = loss / len(self.testset)
#         return loss, acc
    
    
    @torch.no_grad()
    def evaluate(self):
        self.model.eval()
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []
        
        for inputs, targets in self.testset:
            
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            outputs = self.model(inputs)
            loss += self.criterion(outputs, targets)
            predicted = outputs > 0.5
            
#             print("predicted",predicted)
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(targets.cpu().numpy())
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
        
        loss = loss / len(self.testset)
        acc = correct / total
        
        # print("loss: %f\n" % (loss))
        
        print(f"Testing Accuracy: {acc:.2%}")
#         print(predicted_labels," :: ",true_labels)
        precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
        
        recall = recall_score(true_labels, predicted_labels)

        print(f"Precision: {precision:.2%}")
        print(f"Recall: {recall:.2%}")
        
        print("\n\n")

        
        return loss, acc
    

    def train(self):
        print("Base Client train called")
        pass

    def _train(self):
        print("Base Client __train called")
        pass

    def get_client_local_dataset(self):
        
#         datasets = get_dataloader(
#             self.dataset,
#             self.client_id,
#             self.batch_size,
#             self.valset_ratio,
#             self.testset_ratio,
#         )

        print("self.client_id:: ",self.client_id)
                                
#         with open("..\\train_1_client.pkl", "rb") as f:
#             self.trainset = pickle.load(f)
            
        with open("..\\clients_training.pkl", "rb") as f:
            self.trainset = pickle.load(f)
            
        with open("..\\clients_validation.pkl", "rb") as f:
            self.valset = pickle.load(f)

#         with open("..\\valid_1_client.pkl", "rb") as f:
#             self.valset  = pickle.load(f)
        
        with open("..\\clients_testing.pkl", "rb") as f:
            self.testset  = pickle.load(f)
        
        self.trainset = self.trainset[self.client_id]
        self.valset = self.valset[self.client_id]       
        self.testset = self.testset[self.client_id]

                
        print("self.client_id :: len(self.trainset)",self.client_id,"::",len(self.trainset))
        
#         self.trainset = datasets["train"]
#         self.valset = datasets["val"]        
#         self.testset = datasets["test"]

    def _log_while_training(self, evaluate=True, verbose=False):
        def _log_and_train(*args, **kwargs):
            loss_before = 0
            loss_after = 0
            acc_before = 0
            acc_after = 0
            if evaluate:
                loss_before, acc_before = self.evaluate()

            res = self._train(*args, **kwargs)

            if evaluate:
                loss_after, acc_after = self.evaluate()
            
            
#             if verbose:
#                 self.logger.log(
#                     "client [{}]   [bold red]loss: {:.4f} -> {:.4f}    [bold blue]accuracy: {:.2f}% -> {:.2f}%".format(
#                         self.client_id, loss_before, loss_after, acc_before, acc_after
#                     )
#                 )

            eval_stats = {
                "loss_before": loss_before,
                "loss_after": loss_after,
                "acc_before": acc_before,
                "acc_after": acc_after,
            }
            
            return res, eval_stats

        return _log_and_train

    def set_parameters(self, model_params: OrderedDict):

        self.model.load_state_dict(model_params, strict=True)


In [5]:

class FedAvgClient(ClientBase):
    def __init__(
        self,
        backbone: torch.nn.Module,
        dataset: str,
        batch_size: int,
        valset_ratio: float,
        testset_ratio: float,
        local_epochs: int,
        local_lr: float,
        logger: Console,
        gpu: int,
    ):
        super(FedAvgClient, self).__init__(
            backbone,
            dataset,
            batch_size,
            valset_ratio,
            testset_ratio,
            local_epochs,
            local_lr,
            logger,
            gpu,
        )

    def train(
        self,
        client_id: int,
        model_params: OrderedDict[str, torch.Tensor],
        verbose=True,
    ):
        self.client_id = client_id
        self.set_parameters(model_params)
        self.get_client_local_dataset()
        self.model.to(self.device)
        res, stats = self._log_while_training(evaluate=True, verbose=verbose)()
        self.model.cpu()
        return res, stats

    def _train(self):
        self.model.train()
        for _ in range(self.local_epochs):
            for x, y in self.trainset:
                x, y = x.to(self.device), y.to(self.device)
                logits = self.model(x)
                loss = self.criterion(logits, y)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
        return list(self.model.state_dict().values()), len(self.trainset.dataset)

    def test(
        self, client_id: int, model_params: OrderedDict[str, torch.Tensor],
    ):
        self.client_id = client_id
        self.set_parameters(model_params)
        self.get_client_local_dataset()
        self.model.to(self.device)
        loss, acc = self.evaluate()
        self.model.cpu()
        stats = {"loss": loss, "acc": acc}
        return stats


In [6]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(16, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [7]:
class ServerBase:
    def __init__(self, args: Namespace, algo: str):
        self.algo = algo
        self.args = args
        
        # default log file format
        self.log_name = "{}_{}_{}_{}.html".format(
            self.algo,
            self.args.dataset,
            self.args.global_epochs,
            self.args.local_epochs,
        )

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        fix_random_seed(self.args.seed)
        
#         self.backbone= (DeepNet)   
        
        self.backbone = (
            CNNWithBatchNorm
            if self.args.dataset in ["cifar10", "cifar100"]
            else DeepNet
        )
        
        print("self.backbone",self.backbone)
        
        self.logger = Console(record=True, log_path=False, log_time=False,)
        
        self.client_id_indices, self.client_num_in_total = [0,1,2,3,4,5,6,7,8,9], 10                
        
        self.temp_dir ="D:/Download/pythonProject/HiWi/pFedLA_Folktable/temp/my_model/"

#         name_of_model = '../../WW_WM_BW.pth'
        #  self.temp_dir = TEMP_DIR / self.algo
        if not os.path.isdir(self.temp_dir):
            os.makedirs(self.temp_dir)
            
#         _dummy_model = self.backbone(self.args.dataset)
        _dummy_model = self.backbone()
#         _dummy_model = DeepNet()
    
#         print("_dummy_model:: base",_dummy_model)
        
        passed_epoch = 0
        self.global_params_dict: OrderedDict[str : torch.Tensor] = None
        
        if os.listdir(self.temp_dir) != []:
            if os.path.exists(os.path.join(self.temp_dir, "global_model.pt")):
                
                self.global_params_dict = torch.load(os.path.join(self.temp_dir, "global_model.pt"))
                self.logger.log("Find existed global model...")

            if os.path.exists(os.path.join(self.temp_dir, "epoch.pkl")):  
                with open(os.path.join(self.temp_dir, "epoch.pkl"), "rb") as f:
#               
                    passed_epoch = pickle.load(f)
                self.logger.log(f"Have run {passed_epoch} epochs already.",)
        else:
            self.global_params_dict = OrderedDict(_dummy_model.state_dict())
            
#         self.global_params_dict = OrderedDict(_dummy_model.state_dict())
#         print("self.global_params_dict: ",self.global_params_dict)

        self.global_epochs = self.args.global_epochs - passed_epoch
#         self.global_epochs = self.global_epochs - passed_epoch
    
        self.logger.log("Backbone:", _dummy_model)

        self.trainer: ClientBase = None
        self.all_clients_stats = {i: {} for i in self.client_id_indices}
           
            
        print("Base Server Called !")
        
    def train(self):

        print("In Base server  train\n")
        
        self.logger.log("=" * 30, "TRAINING", "=" * 30, style="bold green")
        progress_bar = (
            track(
                range(self.global_epochs),
                "[bold green]Training...",
                console=self.logger,
            )
            if not self.args.log
            else tqdm(range(self.global_epochs), "Training...")
        )
        for E in progress_bar:

            if E % self.args.verbose_gap == 0:
                self.logger.log("=" * 30, f"ROUND: {E}", "=" * 30)

            selected_clients = random.sample(
                self.client_id_indices, self.args.client_num_per_round
            )
            
            updated_params_cache = []
            weights_cache = []

            for client_id in selected_clients:
                
                client_local_params = clone_parameters(self.global_params_dict)
                                
                (updated_params, weight), stats = self.trainer.train(
                    client_id=client_id,
                    model_params=client_local_params,
                    verbose=(E % self.args.verbose_gap) == 0,
                )

                updated_params_cache.append(updated_params)
                weights_cache.append(weight)
                self.all_clients_stats[client_id][f"ROUND: {E}"] = (
                    f"{stats['loss_before']:.4f} -> {stats['loss_after']:.4f}",
                )

            self.aggregate_parameters(updated_params_cache, weights_cache)

            if E % self.args.save_period == 0:
                torch.save(
                    self.global_params_dict, os.path.join(self.temp_dir, "global_model.pt"),
                )
                with open(os.path.join(self.temp_dir, "epoch.pkl"), "wb") as f:
                    pickle.dump(E, f)
        self.logger.log(self.all_clients_stats)
    
    @torch.no_grad()
    def aggregate_parameters(self, updated_params_cache, weights_cache):
        weight_sum = sum(weights_cache)
        weights = torch.tensor(weights_cache, device=self.device) / weight_sum

        aggregated_params = []

        for params in zip(*updated_params_cache):
            aggregated_params.append(
                torch.sum(weights * torch.stack(params, dim=-1), dim=-1)
            )

        self.global_params_dict = OrderedDict(
            zip(self.global_params_dict.keys(), aggregated_params)
        )

                
    def test(self) -> None:
        self.logger.log("=" * 30, "TESTING", "=" * 30, style="bold blue")
        all_loss = []
        all_acc = []
        for client_id in track(
            self.client_id_indices,
            "[bold blue]Testing...",
            console=self.logger,
            disable=self.args.log,
        ):
            client_local_params = clone_parameters(self.global_params_dict)
            stats = self.trainer.test(
                client_id=client_id, model_params=client_local_params,
            )

            self.logger.log(
                f"client [{client_id}]  [red]loss: {stats['loss']:.4f}    [magenta]accuracy: {stats['acc']:.2f}%"
            )
            all_loss.append(stats["loss"])
            all_acc.append(stats["acc"])

        self.logger.log("=" * 20, "RESULTS", "=" * 20, style="bold green")
        self.logger.log(
            "loss: {:.4f}    accuracy: {:.2f}%".format(
                sum(all_loss) / len(all_loss), sum(all_acc) / len(all_acc),
            )
        )        
    
    def run(self):
        print("Base server Run called")
        self.logger.log("Arguments:", dict(self.args._get_kwargs()))
        self.train()
        self.test()
        if self.args.log:
            if not os.path.isdir(LOG_DIR):
                os.mkdir(LOG_DIR)
            self.logger.save_html(LOG_DIR / self.log_name)

        # delete all temporary files
#         if os.listdir(self.temp_dir) != []:
#             os.system(f"rm -rf {self.temp_dir}")


In [8]:
args1 = Namespace(
    k=2,
    global_epochs=3,
    local_epochs=15,
    local_lr=0.001,
    hn_lr=0.002,
    verbose_gap=20,
    embedding_dim=200,
    hidden_dim=200,
    dataset="no",
    batch_size=64,
    valset_ratio=0.0,
    testset_ratio=0.3,
    gpu=True,
    log=0,
    seed=5,
    client_num_per_round=2,
    save_period=10,
)


In [9]:
class FedAvgServer(ServerBase):
    def __init__(self):
        super(FedAvgServer, self).__init__(args1, "FedAvg")

        self.trainer = FedAvgClient(
            backbone=self.backbone(),
            dataset=self.args.dataset,
            batch_size=self.args.batch_size,
            valset_ratio=self.args.valset_ratio,
            testset_ratio=self.args.testset_ratio,
            local_epochs=self.args.local_epochs,
            local_lr=self.args.local_lr,
            logger=self.logger,
            gpu=self.args.gpu,
        )


In [10]:
# FedAvg and FairFed

In [11]:
server = FedAvgServer()

self.backbone <class '__main__.DeepNet'>


Backbone: DeepNet(                                                                                                 
  (layer1): Linear(in_features=16, out_features=512, bias=True)                                                    
  (act1): ReLU()                                                                                                   
  (dropout1): Dropout(p=0.5, inplace=False)                                                                        
  (layer2): Linear(in_features=512, out_features=256, bias=True)                                                   
  (act2): ReLU()                                                                                                   
  (layer3): Linear(in_features=256, out_features=60, bias=True)                                                    
  (act3): ReLU()                                                                                                   
  (output): Linear(in_features=60, out_features=1, bias=True)                                                      
  (sigmoid): Sigmoid()                                                                                             
)

Base Server Called !
Base Client constructor


In [12]:
server.run()

Base server Run called


Arguments:                                                                                                         
{                                                                                                                  
    'k': 2,                                                                                                        
    'global_epochs': 3,                                                                                            
    'local_epochs': 15,                                                                                            
    'local_lr': 0.001,                                                                                             
    'hn_lr': 0.002,                                                                                                
    'verbose_gap': 20,                                                                                             
    'embedding_dim': 200,                                                                                          
    'hidden_dim': 200,                                                                                             
    'dataset': 'no',                                                                                               
    'batch_size': 64,                                                                                              
    'valset_ratio': 0.0,                                                                                           
    'testset_ratio': 0.3,                                                                                          
    'gpu': True,                                                                                                   
    'log': 0,                                                                                                      
    'seed': 5,                                                                                                     
    'client_num_per_round': 2,                                                                                     
    'save_period': 10                                                                                              
}

In Base server  train



============================== TRAINING ==============================                                             

Output()

...

Precision: 70.36%

Recall: 78.71%

client [9]  loss: 0.4476    accuracy: 0.78%

==================== RESULTS ====================                                                                  

loss: 0.4318    accuracy: 0.79%

In [1]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    correct = 0
    total = 0
    loss=0
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        loss += criterion(outputs, targets)
        
        predicted = outputs > 0.5
                
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
    print("Correct: ",correct,"Total: ", total)
    
    average_loss = loss / len(test_loader)
    accuracy = correct / total
    
    print(f"Testing Accuracy: {accuracy:.2%}")
    print(f"Testing Loss: " ,average_loss)

NameError: name 'model' is not defined

In [25]:
with open("D:/Download/pythonProject/HiWi/pFedLA_Folktable/src/pkl_files/hn/0.pkl", 'rb') as f:
    qw = pickle.load(f)
    
qw["mlp"][0].weight

Parameter containing:
tensor([[ 0.0273,  0.0562, -0.0342,  ...,  0.0298, -0.0126, -0.0052],
        [ 0.0328, -0.0955, -0.1000,  ..., -0.0489,  0.0381,  0.0211],
        [-0.0320,  0.0745, -0.0512,  ..., -0.0103,  0.0453, -0.0010],
        ...,
        [ 0.0384, -0.0118,  0.0080,  ..., -0.0856, -0.0647, -0.0319],
        [ 0.0429,  0.0723,  0.0675,  ..., -0.0407, -0.0915,  0.0258],
        [-0.0868, -0.0254, -0.0313,  ..., -0.0187, -0.0333,  0.0142]],
       requires_grad=True)

In [9]:
# D:\Download\pythonProject\HiWi\pFedLA_Folktable\temp\hn
with open("D:/Download/pythonProject/HiWi/pFedLA-master_original/temp/hn/8.pkl", 'rb') as f:
    a1 = pickle.load(f)
    
a1["mlp"][0].weight

Parameter containing:
tensor([[-0.0937, -0.0340, -0.0789,  ...,  0.0953, -0.0907,  0.0428],
        [-0.0973, -0.0327,  0.0957,  ..., -0.0736,  0.0715,  0.0400],
        [ 0.0272, -0.0741, -0.0024,  ...,  0.0134,  0.0945, -0.0741],
        ...,
        [-0.0915,  0.0650, -0.0139,  ..., -0.0919,  0.0284, -0.0107],
        [ 0.0486, -0.0921, -0.0964,  ...,  0.0980, -0.0358,  0.0721],
        [ 0.0323, -0.0272,  0.0640,  ..., -0.0886,  0.0630,  0.0817]],
       requires_grad=True)

In [32]:
# D:\Download\pythonProject\HiWi\pFedLA_Folktable\temp\hn
with open("D:/Download/pythonProject/HiWi/pFedLA-master_original/temp/hn/8.pkl", 'rb') as f:
    a2 = pickle.load(f)
    
a2["mlp"][0].weight[0]

tensor([-0.0937, -0.0340, -0.0789,  0.0736, -0.0949,  0.0642,  0.0790,  0.0037,
        -0.0856,  0.0770, -0.0534, -0.0941,  0.0115,  0.0501, -0.0512,  0.0796,
        -0.0407,  0.0587, -0.0320, -0.0691,  0.0732,  0.0862, -0.0808,  0.0117,
        -0.0041, -0.0942,  0.0254, -0.0815, -0.0757,  0.0142, -0.0595, -0.0525,
        -0.0617, -0.0892, -0.0879, -0.0020, -0.0862,  0.0740, -0.0417,  0.0953,
         0.0618,  0.0115, -0.0543,  0.0179, -0.0213, -0.0651,  0.0661,  0.0287,
        -0.0900, -0.0155,  0.0535, -0.0633, -0.0122,  0.0616,  0.0409, -0.0100,
        -0.0223, -0.0163,  0.0374,  0.0285, -0.0653,  0.0042, -0.0179, -0.0476,
         0.0977,  0.0163,  0.0297, -0.0257, -0.0303, -0.0456, -0.0347, -0.0898,
         0.0068, -0.0229, -0.0901, -0.0588,  0.0831, -0.0576, -0.0739,  0.0262,
         0.0600,  0.0905, -0.0072, -0.0443,  0.0981,  0.0350, -0.0329, -0.0846,
        -0.0869,  0.0158,  0.0663, -0.0791,  0.0413, -0.0424, -0.0297, -0.0697,
        -0.0594,  0.0953, -0.0907,  0.04

In [33]:
torch.equal(a1["mlp"][0].weight[0],a2["mlp"][0].weight[0])

True

In [2]:
import torch

# Create a 3x3x3 tensor
tensor = torch.tensor([
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],
    
    [[10, 11, 12],
     [13, 14, 15],
     [16, 17, 18]],
    
    [[19, 20, 21],
     [22, 23, 24],
     [25, 26, 27]]
])

# Calculate sum along the last dimension (dim=-1)
sum_along_last_dim = torch.sum(tensor, dim=-1)

print(sum_along_last_dim)


tensor([[ 6, 15, 24],
        [33, 42, 51],
        [60, 69, 78]])
